In [1]:
# General Stuff:
import numpy as np
import pandas as pd

# Our Stuff:
from models.AutoRecBase import AutoRecBase
from scripts.get_data import download_2_data_sets, ratings_to_train_test

import torch
from torch import nn
import pytorch_lightning as pl

# Visualization Stuff
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 240)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
if torch.cuda.is_available():
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# AutoRec - AutoEncoders Meet Collaborative Filtering - PyTorch

<a id="toc"></a>
## Table of Content
1. [Introduction](#introduction)
1. [Conclusions](#conclusions)

<a id="introduction"></a>

## Introduction

In this notebook we will review a collaborative filtering approach using autoencoders, as suggested by Sedhain et al, in their 2015 paper "AutoRec: Autoencoders Meet Collaborative Filtering".

We will then introduce several improvements and asses them.

[Table of content](#toc)

In [2]:
download_2_data_sets()

/Users/odedgolden/Developer/AutoRec/scripts/get_data.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(


In [3]:
train_loader, val_loader = ratings_to_train_test(1,0, 1,10)

In [6]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs

Launching TensorBoard...

Go to:  [TensorBoard](http://localhost:6006)

In [5]:
model = AutoRecBase(3706,
                    25,
                    nn.PReLU,
                    nn.PReLU,
                    nn.MSELoss
                    )
# training
trainer = pl.Trainer(gpus=0,max_epochs=10)
trainer.fit(model,train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | encoder   | Linear  | 92.7 K
1 | act_1     | PReLU   | 1     
2 | decoder   | Linear  | 96.4 K
3 | act_2     | PReLU   | 1     
4 | loss_func | MSELoss | 0     
--------------------------------------
189 K     Trainable params
0         Non-trainable params
189 K     Total params
0.756     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
